In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkDates").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://amazon-ebook-reviews.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True)
df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1S32NOGVO5CTV|          5|            0|          0|   N|                Y|
|R31SVA4OED5Z6D|          4|            0|          0|   N|                Y|
|R14EAH3P2CYZJG|          5|            0|          0|   N|                Y|
|R22SLL34YR3BT2|          5|            1|          1|   N|                Y|
|R2U4CSZ5SIA402|          5|            2|          2|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
votes_20plus_df = df.filter("total_votes >= 20")
votes_20plus_df.count(), votes_20plus_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R2XES7LGRQZFH3|          5|           42|         42|   N|                Y|
|R2H2KND29Y0VBN|          5|           20|         22|   N|                Y|
|R3FBLYZ2CSWCJE|          5|           72|         75|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



(7055, None)

In [11]:
votes_helptotot_halfplus_df = votes_20plus_df.filter("helpful_votes/total_votes >= 0.5")
votes_helptotot_halfplus_df.count(), votes_helptotot_halfplus_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R2XES7LGRQZFH3|          5|           42|         42|   N|                Y|
|R2H2KND29Y0VBN|          5|           20|         22|   N|                Y|
|R3FBLYZ2CSWCJE|          5|           72|         75|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



(6711, None)

In [13]:
votes_helptotot_halfplus_vine_df = votes_helptotot_halfplus_df.filter("vine == 'Y'")
votes_helptotot_halfplus_vine_df.count(), votes_helptotot_halfplus_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



(21, None)

In [14]:
votes_helptotot_halfplus_notvine_df = votes_helptotot_halfplus_df.filter("vine == 'N'")
votes_helptotot_halfplus_notvine_df.count(), votes_helptotot_halfplus_notvine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R2XES7LGRQZFH3|          5|           42|         42|   N|                Y|
|R2H2KND29Y0VBN|          5|           20|         22|   N|                Y|
|R3FBLYZ2CSWCJE|          5|           72|         75|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



(6690, None)

In [17]:
vine_total = votes_helptotot_halfplus_vine_df.count()
vine_5star = votes_helptotot_halfplus_vine_df.filter("star_rating == 5").count()
vine_5star, vine_total, vine_5star/vine_total*100

(10, 21, 47.61904761904761)

In [18]:
notvine_total = votes_helptotot_halfplus_notvine_df.count()
notvine_5star = votes_helptotot_halfplus_notvine_df.filter("star_rating == 5").count()
notvine_5star, notvine_total, notvine_5star/notvine_total*100

(3448, 6690, 51.53961136023917)